In [4]:
# Import dependencies
import matplotlib
from matplotlib import style
style.use('fivethirtyeight')
import matplotlib.pyplot as plt
import pandas as pd

# Python SQL toolkit and Object Relational Mapper
from sqlalchemy import create_engine, text

In [5]:
# load the full database (this database is created via create_db.ipynb from the csv files in the Resources directory which need to be downloaded from Kaggle)
engine = create_engine("sqlite:///open_university.sqlite", echo=False)

## Load the data

The plan is to just look at the interaction with the course material to see if the student will be able to predict if the student will pass or fail.

Not including information about the background of the student (demographics) or the scores of the student assessments but include if they submit the assessments or not.

We are going to filter out students that withdraw from the course before the start of the course.

In [6]:
# load a subset of the student data
student_ds = engine.execute(text("""
SELECT sI.code_module, sI.code_presentation, sI.id_student, sI.final_result, sR.date_registration, sR.date_unregistration
FROM studentInfo as sI
LEFT JOIN studentRegistration as sR ON sI.id_student = sR.id_student
WHERE NOT (sR.date_unregistration <= -11 AND sI.final_result = 'Withdrawn')
""")).fetchall()

# Remove all students that withdraw before 11 days before the course
# these students would not have interacted with the course material

student_df = pd.DataFrame(student_ds, columns=['code_module', 'code_presentation', 'id_student', 'final_result', 'date_registration', 'date_unregistration'])
student_df = student_df.astype({'code_module':'string', 'code_presentation':'string', 'id_student':'string', 'final_result':'string'})
student_df.head()

2024-05-12 12:33:13,563 INFO sqlalchemy.engine.Engine 
SELECT sI.code_module, sI.code_presentation, sI.id_student, sI.final_result, sR.date_registration, sR.date_unregistration
FROM studentInfo as sI
LEFT JOIN studentRegistration as sR ON sI.id_student = sR.id_student
WHERE NOT (sR.date_unregistration <= -11 AND sI.final_result = 'Withdrawn')

2024-05-12 12:33:13,563 INFO sqlalchemy.engine.Engine [generated in 0.00084s] ()


In [20]:
student_df.info()
# view the number of unique values in each of the fields
display(student_df.nunique())
# have a closer look at the final results
display(student_df['final_result'].value_counts())
student_pass    = student_df['final_result'].loc[(student_df['final_result'] == 'Distinction') | (student_df['final_result'] == 'Pass')].value_counts().sum()
student_results = student_df['final_result'].value_counts().sum()
print(f"{student_pass} passing grades over {student_results} total grades in 7 courses over the 4 presentations of the courses")
print(f"This is telling us that some of the students participated in more than one course over the 4 presentations")

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 36688 entries, 0 to 36687
Data columns (total 6 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   code_module          36688 non-null  string 
 1   code_presentation    36688 non-null  string 
 2   id_student           36688 non-null  string 
 3   final_result         36688 non-null  string 
 4   date_registration    36666 non-null  float64
 5   date_unregistration  11592 non-null  float64
dtypes: float64(2), string(4)
memory usage: 1.7 MB


code_module                7
code_presentation          4
id_student             27295
final_result               4
date_registration        320
date_unregistration      355
dtype: int64

final_result
Pass           14682
Withdrawn       9904
Fail            8478
Distinction     3624
Name: count, dtype: Int64

18306 passing grades over 36688 total grades in 7 courses over the 4 presentations of the courses
This is telling us that some of the students participated in more than one course over the 4 presentations


* I removed students that Withdrew before 11 days before the presentation starts because they whould not have participated in the course.
* It looks like out of the 27,295 students there were 36,388 registrations which means some students where registered more than once. Of all the registrations there where 11,592 unregistrations.
* 


In [5]:
bf_course_df = student_df.loc[student_df['date_unregistration'] <= -11].copy()
display(bf_course_df)
display(bf_course_df.describe())
#display(student_df.loc[student_df['date_registration'].isnull()])
bf_course_df.info()

,code_module,code_presentation,id_student,final_result,date_registration,date_unregistration
716,AAA,2014J,2318055,Pass,-56.0,-19.0
987,BBB,2013B,335910,Fail,-184.0,-43.0
1816,BBB,2013B,543356,Pass,-52.0,-23.0
2812,BBB,2013J,393327,Pass,-128.0,-109.0
3271,BBB,2013J,546941,Distinction,-53.0,-30.0
...,...,...,...,...,...,...
35251,GGG,2014B,542562,Pass,-225.0,-212.0
35253,GGG,2014B,542562,Pass,-85.0,-73.0
35258,GGG,2014B,548578,Fail,-89.0,-87.0
35639,GGG,2014B,625903,Pass,-78.0,-18.0


,date_registration,date_unregistration
count,246.000000,252.000000
mean,-117.556911,-66.781746
std,59.090201,50.987812
min,-320.000000,-365.000000
25%,-148.000000,-87.500000
50%,-115.000000,-53.000000
75%,-78.000000,-26.000000
max,-17.000000,-11.000000


<class 'pandas.core.frame.DataFrame'>
Index: 252 entries, 716 to 35732
Data columns (total 6 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   code_module          252 non-null    string 
 1   code_presentation    252 non-null    string 
 2   id_student           252 non-null    string 
 3   final_result         252 non-null    string 
 4   date_registration    246 non-null    float64
 5   date_unregistration  252 non-null    float64
dtypes: float64(2), string(4)
memory usage: 13.8 KB


In [6]:
bf_course_df['code_presentation'].unique()

<StringArray>
['2014J', '2013B', '2013J', '2014B']
Length: 4, dtype: string

## Student Interaction with course material

In [7]:
## need to add the Vle connected to Student Vle
vle_ds = engine.execute(text("""
SELECT cast(vle.id_site as text), vle.code_module, vle.code_presentation, vle.activity_type, vle.week_from, vle.week_to, cast(sVle.id_student as text), sVle.date, SUM(sVLe.sum_click) as sum_click
FROM vle
LEFT JOIN studentVle AS sVle ON vle.id_site = sVle.id_site AND vle.code_presentation = sVle.code_presentation AND vle.code_module = sVle.code_module
GROUP BY vle.code_module, vle.code_presentation, sVle.id_student
""")).fetchall()
vle_df = pd.DataFrame(vle_ds, columns=['id_site', 'code_module', 'code_presentation', 'activity_type', 'week_from', 'week_to', 'id_student', 'date', 'sum_click'])
vle_df = vle_df.astype({'id_site':'string', 'code_module':'string', 'code_presentation':'string', 'activity_type':'string', 'id_student':'string'})

In [8]:
display(vle_df)
display(vle_df.nunique())
vle_df.info()

,id_site,code_module,code_presentation,activity_type,week_from,week_to,id_student,date,sum_click
0,546897,AAA,2013J,url,NaN,NaN,<NA>,NaN,NaN
1,546614,AAA,2013J,homepage,NaN,NaN,11391,-5.0,934.0
2,546614,AAA,2013J,homepage,NaN,NaN,28400,-10.0,1435.0
3,546614,AAA,2013J,homepage,NaN,NaN,30268,-10.0,281.0
4,546614,AAA,2013J,homepage,NaN,NaN,31604,-10.0,2158.0
...,...,...,...,...,...,...,...,...,...
29240,897051,GGG,2014J,resource,2.0,2.0,2640965,17.0,41.0
29241,896962,GGG,2014J,oucontent,NaN,NaN,2645731,110.0,893.0
29242,896956,GGG,2014J,quiz,NaN,NaN,2648187,149.0,312.0
29243,897051,GGG,2014J,resource,2.0,2.0,2679821,-6.0,275.0


id_site                623
code_module              7
code_presentation        4
activity_type           15
week_from               24
week_to                 24
id_student           26074
date                   281
sum_click             5380
dtype: int64

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 29245 entries, 0 to 29244
Data columns (total 9 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   id_site            29245 non-null  string 
 1   code_module        29245 non-null  string 
 2   code_presentation  29245 non-null  string 
 3   activity_type      29245 non-null  string 
 4   week_from          4705 non-null   float64
 5   week_to            4705 non-null   float64
 6   id_student         29228 non-null  string 
 7   date               29228 non-null  float64
 8   sum_click          29228 non-null  float64
dtypes: float64(4), string(5)
memory usage: 2.0 MB


## Student Assessment

In [9]:
ass_ds = engine.execute(text("""
SELECT cast(sAss.id_student as text), ass.code_module, ass.code_presentation, cast(sAss.id_assessment as text), sAss.date_submitted, ass.date
FROM studentAssessment as sAss
LEFT JOIN assessments as ass ON sAss.id_assessment = ass.id_assessment
""")).fetchall()

assessment_df = pd.DataFrame(ass_ds, columns=['id_student', 'code_module', 'code_presentation', 'id_assessent', 'date_submitted', 'date'])
assessment_df = assessment_df.astype({'id_student':'string', 'code_module':'string', 'code_presentation':'string', 'id_assessent':'string'})
display(assessment_df.describe())
assessment_df.info()

,date_submitted,date
count,173912.000000,171047.000000
mean,116.032942,130.605623
std,71.484148,78.025175
min,-11.000000,12.000000
25%,51.000000,54.000000
50%,116.000000,129.000000
75%,173.000000,214.000000
max,608.000000,261.000000


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 173912 entries, 0 to 173911
Data columns (total 6 columns):
 #   Column             Non-Null Count   Dtype  
---  ------             --------------   -----  
 0   id_student         173912 non-null  string 
 1   code_module        173912 non-null  string 
 2   code_presentation  173912 non-null  string 
 3   id_assessent       173912 non-null  string 
 4   date_submitted     173912 non-null  int64  
 5   date               171047 non-null  float64
dtypes: float64(1), int64(1), string(4)
memory usage: 8.0 MB


We are looking at the students that haven't haded in any assessments.

* We don't want them to skew our data as they never participated.
* There could be reasons but for the momst part the students either withdrew or failed.
* over 75% had unregistered by the 27th day of the presetation.
* It could only be assumed that the 2 that passed had prior learning or some other reason

In [10]:
students_assessed = list(assessment_df['id_student'].unique())
all_students = list(student_df['id_student'].unique())
display(f"Number of students assessed/total students: {len(list(students_assessed))}/{len(list(all_students))}")

student_not_assessed_df = student_df[~student_df['id_student'].isin(students_assessed)].copy()
display(student_not_assessed_df.nunique())
display(student_not_assessed_df['final_result'].value_counts())
display(student_not_assessed_df[['date_registration', 'date_unregistration']].describe())


'Number of students assessed/total students: 23369/27295'

code_module               7
code_presentation         4
id_student             3948
final_result              3
date_registration       227
date_unregistration     189
dtype: int64

final_result
Withdrawn    3480
Fail         1334
Pass            2
Name: count, dtype: Int64

,date_registration,date_unregistration
count,4801.000000,3535.000000
mean,-70.581546,22.876945
std,48.268486,42.463059
min,-309.000000,-232.000000
25%,-104.000000,0.000000
50%,-58.000000,12.000000
75%,-30.000000,27.000000
max,37.000000,240.000000


## The Students that have been assessed.

In [11]:
students_assessed_df = student_df[student_df['id_student'].isin(students_assessed)]
display(students_assessed_df.nunique())
display(students_assessed_df['final_result'].value_counts())
display(students_assessed_df[['date_registration', 'date_unregistration']].describe())

code_module                7
code_presentation          4
id_student             23347
final_result               4
date_registration        317
date_unregistration      354
dtype: int64

final_result
Pass           14680
Fail            7144
Withdrawn       6424
Distinction     3624
Name: count, dtype: Int64

,date_registration,date_unregistration
count,31865.000000,8057.000000
mean,-68.181861,95.841256
std,49.308932,72.560183
min,-320.000000,-365.000000
25%,-99.000000,38.000000
50%,-53.000000,95.000000
75%,-29.000000,154.000000
max,167.000000,444.000000


## The Students that have not been assessed but interacted.

In [12]:

student_not_assessed_interacted_df = vle_df[~vle_df['id_student'].isin(students_assessed)]
display(student_df[student_df['id_student'].isin(list(student_not_assessed_interacted_df['id_student']))].nunique())
display(student_df[student_df['id_student'].isin(list(student_not_assessed_interacted_df['id_student']))]['final_result'].value_counts())

code_module               7
code_presentation         4
id_student             2634
final_result              3
date_registration       205
date_unregistration     183
dtype: int64

final_result
Withdrawn    2337
Fail          998
Pass            1
Name: count, dtype: Int64

## The Students that have been assessed but not interacted.

In [13]:
student_assessed_not_interacted_df = student_df[(student_df['id_student'].isin(students_assessed) & ~student_df['id_student'].isin(list(vle_df['id_student'].unique())))].copy()

display(student_assessed_not_interacted_df)
display(student_assessed_not_interacted_df.nunique())
display(student_df[student_df['id_student'].isin(list(student_assessed_not_interacted_df['id_student']))]['final_result'].value_counts())

,code_module,code_presentation,id_student,final_result,date_registration,date_unregistration
1493,BBB,2013B,517853,Fail,-50.0,NaN
2752,BBB,2013J,355591,Withdrawn,-176.0,111.0
2850,BBB,2013J,415698,Withdrawn,-87.0,76.0
2892,BBB,2013J,440751,Withdrawn,-127.0,-8.0
2948,BBB,2013J,481448,Fail,-30.0,NaN
3268,BBB,2013J,546195,Fail,-10.0,NaN
3540,BBB,2013J,574810,Withdrawn,-29.0,48.0
4562,BBB,2013J,2346025,Fail,-71.0,NaN
4640,BBB,2013J,2650236,Fail,-80.0,NaN
4654,BBB,2014B,38941,Fail,-24.0,NaN


code_module             4
code_presentation       4
id_student             23
final_result            3
date_registration      22
date_unregistration     6
dtype: int64

final_result
Fail         15
Withdrawn     6
Pass          2
Name: count, dtype: Int64

In [42]:
ou_df = pd.merge(pd.merge(student_df,assessment_df,how='left'), vle_df, how='left')

# remove students that do not hand in assessments only one passed so almost 100% fail or withdrawn
ou_df = ou_df[~ou_df['id_student'].isin(student_not_assessed_interacted_df)]

display(ou_df)
display(ou_df.nunique())
ou_df.info()

,code_module,code_presentation,id_student,final_result,date_registration,date_unregistration,id_assessent,date_submitted,date,id_site,activity_type,week_from,week_to,sum_click
0,AAA,2013J,11391,Pass,-159.0,NaN,1752,18.0,19.0,<NA>,<NA>,NaN,NaN,NaN
1,AAA,2013J,11391,Pass,-159.0,NaN,1753,53.0,54.0,<NA>,<NA>,NaN,NaN,NaN
2,AAA,2013J,11391,Pass,-159.0,NaN,1754,115.0,117.0,<NA>,<NA>,NaN,NaN,NaN
3,AAA,2013J,11391,Pass,-159.0,NaN,1755,164.0,166.0,<NA>,<NA>,NaN,NaN,NaN
4,AAA,2013J,11391,Pass,-159.0,NaN,1756,212.0,215.0,<NA>,<NA>,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
209570,GGG,2014J,2684003,Distinction,-28.0,NaN,37437,169.0,173.0,<NA>,<NA>,NaN,NaN,NaN
209571,GGG,2014J,2684003,Distinction,-28.0,NaN,37438,73.0,229.0,<NA>,<NA>,NaN,NaN,NaN
209572,GGG,2014J,2684003,Distinction,-28.0,NaN,37439,150.0,229.0,<NA>,<NA>,NaN,NaN,NaN
209573,GGG,2014J,2684003,Distinction,-28.0,NaN,37440,172.0,229.0,<NA>,<NA>,NaN,NaN,NaN


code_module                7
code_presentation          4
id_student             27295
final_result               4
date_registration        320
date_unregistration      355
id_assessent             188
date_submitted           312
date                      72
id_site                  164
activity_type             10
week_from                 16
week_to                   16
sum_click                917
dtype: int64

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 209575 entries, 0 to 209574
Data columns (total 14 columns):
 #   Column               Non-Null Count   Dtype  
---  ------               --------------   -----  
 0   code_module          209575 non-null  string 
 1   code_presentation    209575 non-null  string 
 2   id_student           209575 non-null  string 
 3   final_result         209575 non-null  string 
 4   date_registration    209524 non-null  float64
 5   date_unregistration  29902 non-null   float64
 6   id_assessent         203702 non-null  string 
 7   date_submitted       203702 non-null  float64
 8   date                 199684 non-null  float64
 9   id_site              1923 non-null    string 
 10  activity_type        1923 non-null    string 
 11  week_from            400 non-null     float64
 12  week_to              400 non-null     float64
 13  sum_click            1923 non-null    float64
dtypes: float64(7), string(7)
memory usage: 22.4 MB


In [15]:
## Defining the categories:
categories = pd.Series(['Very Early', 'Early', 'Late'])

## Applying these categories to the final dataframe:
ou_df['registration'] = pd.cut(ou_df.date_registration, bins = [-322,-160,0,160], labels=categories)
ou_df.head()

,code_module,code_presentation,id_student,final_result,date_registration,date_unregistration,id_assessent,date_submitted,date,id_site,activity_type,week_from,week_to,sum_click,registration
0,AAA,2013J,11391,Pass,-159.0,NaN,1752,18.0,19.0,<NA>,<NA>,NaN,NaN,NaN,Early
1,AAA,2013J,11391,Pass,-159.0,NaN,1753,53.0,54.0,<NA>,<NA>,NaN,NaN,NaN,Early
2,AAA,2013J,11391,Pass,-159.0,NaN,1754,115.0,117.0,<NA>,<NA>,NaN,NaN,NaN,Early
3,AAA,2013J,11391,Pass,-159.0,NaN,1755,164.0,166.0,<NA>,<NA>,NaN,NaN,NaN,Early
4,AAA,2013J,11391,Pass,-159.0,NaN,1756,212.0,215.0,<NA>,<NA>,NaN,NaN,NaN,Early


In [16]:

ou_df['code_presentation'] = ou_df['code_presentation'].replace(['2013B', '2013J', '2014B', '2014J'], ['1', '2', '3', '4'])
ou_df.head()

,code_module,code_presentation,id_student,final_result,date_registration,date_unregistration,id_assessent,date_submitted,date,id_site,activity_type,week_from,week_to,sum_click,registration
0,AAA,2,11391,Pass,-159.0,NaN,1752,18.0,19.0,<NA>,<NA>,NaN,NaN,NaN,Early
1,AAA,2,11391,Pass,-159.0,NaN,1753,53.0,54.0,<NA>,<NA>,NaN,NaN,NaN,Early
2,AAA,2,11391,Pass,-159.0,NaN,1754,115.0,117.0,<NA>,<NA>,NaN,NaN,NaN,Early
3,AAA,2,11391,Pass,-159.0,NaN,1755,164.0,166.0,<NA>,<NA>,NaN,NaN,NaN,Early
4,AAA,2,11391,Pass,-159.0,NaN,1756,212.0,215.0,<NA>,<NA>,NaN,NaN,NaN,Early


In [17]:
ou_df = ou_df.replace(['Pass', 'Withdrawn', 'Distinction', 'Fail'], ['1', '0', '1', '0'])
ou_df.head()

,code_module,code_presentation,id_student,final_result,date_registration,date_unregistration,id_assessent,date_submitted,date,id_site,activity_type,week_from,week_to,sum_click,registration
0,AAA,2,11391,1,-159.0,NaN,1752,18.0,19.0,<NA>,<NA>,NaN,NaN,NaN,Early
1,AAA,2,11391,1,-159.0,NaN,1753,53.0,54.0,<NA>,<NA>,NaN,NaN,NaN,Early
2,AAA,2,11391,1,-159.0,NaN,1754,115.0,117.0,<NA>,<NA>,NaN,NaN,NaN,Early
3,AAA,2,11391,1,-159.0,NaN,1755,164.0,166.0,<NA>,<NA>,NaN,NaN,NaN,Early
4,AAA,2,11391,1,-159.0,NaN,1756,212.0,215.0,<NA>,<NA>,NaN,NaN,NaN,Early


In [18]:
# find the min and max of the column
print('minimum:', ou_df['sum_click'].min(), '-', ou_df['sum_click'].max())

## Defining the categories label:
categories3 = pd.Series(['0-2295', '2295-4590', '4590-6885', '6885-19179', '9179-11474', '11474-13769'])

## Applying these categories both to the auxiliary and to the working datasets:
ou_df['total_clicks'] = pd.cut(ou_df.sum_click, bins = [0,2295,4590,6885,9179,11474,13769], labels=categories3)
ou_df['total_clicks'].nunique()

minimum: 3.0
maximum: 13769.0


6

In [19]:

del ou_df['date_registration']
del ou_df['id_student']
#del ou_df['code_presentation']
del ou_df['sum_click']
del ou_df['week_to']
del ou_df['date']
#del ou_df['date_submitted']
del ou_df['id_assessent']
del ou_df['date_unregistration']


In [20]:
ou_df['week_from']      = ou_df['week_from'].fillna(1)
ou_df['final_result']   = ou_df['final_result'].fillna(0)
ou_df['date_submitted'] = ou_df['date_submitted'].fillna(ou_df['date_submitted'].max())
ou_df['total_clicks']   = ou_df['total_clicks'].fillna('0-2295')
ou_df['registration']   = ou_df['registration'].fillna('Early')


for c in ou_df.columns:
    ou_df[c] = ou_df[c].astype("str")

ou_df['id_site']        = ou_df['id_site'].replace({'<NA>': '546712'})
ou_df['activity_type']  = ou_df['activity_type'].replace({'<NA>': 'url'})

for c in ou_df.columns:
    print(c)
    display(ou_df[c].unique())

code_module


array(['AAA', 'BBB', 'CCC', 'DDD', 'EEE', 'FFF', 'GGG'], dtype=object)

code_presentation


array(['2', '4', '1', '3'], dtype=object)

final_result


...

array(['1.0', '8.0', '4.0', '19.0', '22.0', '6.0', '11.0', '3.0', '2.0',
       '5.0', '28.0', '29.0', '13.0', '18.0', '27.0', '9.0'], dtype=object)

registration


array(['Early', 'Very Early', 'Late', 'nan'], dtype=object)

total_clicks


array(['0-2295', '6885-19179', '4590-6885', '2295-4590', '9179-11474',
       '11474-13769'], dtype=object)

In [21]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import tensorflow as tf

from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier

from sklearn.svm import SVC, LinearSVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.linear_model import Perceptron
from sklearn.linear_model import SGDClassifier

In [22]:
# get the locations
X = ou_df.iloc[:, :]
y = ou_df.iloc[:, :]
 
# split the dataset
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.30, random_state=0)

In [23]:
X_train.describe(include=['O'])

,code_module,code_presentation,final_result,date_submitted,id_site,activity_type,week_from,registration,total_clicks
count,146702,146702,146702,146702,146702,146702,146702,146702,146702
unique,7,4,2,303,145,10,14,4,6
top,FFF,4,1,608.0,546712,url,1.0,Early,0-2295
freq,44586,45907,108038,4075,145364,145510,146437,139614,146508


In [24]:
combine = [X_train, X_test]

In [25]:
module_converting = {'AAA':1, 'BBB':2, 'CCC':3, 'DDD':4, 'EEE':5, 'FFF':6, 'GGG':7}
for dataset in combine:
    dataset['code_module'] = dataset['code_module'].map(module_converting)

X_train.head()

,code_module,code_presentation,final_result,date_submitted,id_site,activity_type,week_from,registration,total_clicks
76088,3,4,1,31.0,546712,url,1.0,Early,0-2295
183804,6,4,0,100.0,546712,url,1.0,Early,0-2295
160319,6,2,1,237.0,546712,url,1.0,Early,0-2295
176109,6,4,0,608.0,546712,url,1.0,Early,0-2295
30016,2,2,1,168.0,546712,url,1.0,Early,0-2295


In [26]:
module_converting = {'0-2295':1, '2295-4590':2, '4590-6885':3, '6885-19179':4, '9179-11474':5, '11474-13769':6}
for dataset in combine:
    dataset['total_clicks'] = dataset['total_clicks'].map(module_converting)

X_train.head()

,code_module,code_presentation,final_result,date_submitted,id_site,activity_type,week_from,registration,total_clicks
76088,3,4,1,31.0,546712,url,1.0,Early,1
183804,6,4,0,100.0,546712,url,1.0,Early,1
160319,6,2,1,237.0,546712,url,1.0,Early,1
176109,6,4,0,608.0,546712,url,1.0,Early,1
30016,2,2,1,168.0,546712,url,1.0,Early,1


In [27]:

module_converting = {'url':1, 'oucontent':2, 'resource':3, 'forumng':4, 'quiz':5, 'subpage':6,
       'homepage':7, 'oucollaborate':8, 'externalquiz':9, 'ouelluminate':10}
for dataset in combine:
    dataset['activity_type'] = dataset['activity_type'].map(module_converting)
    dataset['activity_type'] = dataset['activity_type'].fillna(1)

X_train.head()

,code_module,code_presentation,final_result,date_submitted,id_site,activity_type,week_from,registration,total_clicks
76088,3,4,1,31.0,546712,1,1.0,Early,1
183804,6,4,0,100.0,546712,1,1.0,Early,1
160319,6,2,1,237.0,546712,1,1.0,Early,1
176109,6,4,0,608.0,546712,1,1.0,Early,1
30016,2,2,1,168.0,546712,1,1.0,Early,1


In [28]:
module_converting = {'Very Early':1, 'Early':2, 'Late':3}
for dataset in combine:
    dataset['registration'] = dataset['registration'].map(module_converting)
    dataset['registration'] = dataset['registration'].fillna(2)

X_train.head()

,code_module,code_presentation,final_result,date_submitted,id_site,activity_type,week_from,registration,total_clicks
76088,3,4,1,31.0,546712,1,1.0,2.0,1
183804,6,4,0,100.0,546712,1,1.0,2.0,1
160319,6,2,1,237.0,546712,1,1.0,2.0,1
176109,6,4,0,608.0,546712,1,1.0,2.0,1
30016,2,2,1,168.0,546712,1,1.0,2.0,1


In [29]:
X_train_r = X_train.drop("final_result", axis=1)
Y_train_r = X_train["final_result"]
X_test_r = X_test.drop("final_result", axis=1).copy()
X_train_r.shape, Y_train_r.shape, X_test_r.shape

((146702, 8), (146702,), (62873, 8))

In [30]:
logreg = LogisticRegression()
logreg.fit(X_train_r, Y_train_r)
Y_pred = logreg.predict(X_test_r)
acc_log = round(logreg.score(X_train_r, Y_train_r) * 100, 2)
acc_log

73.64

In [31]:
coeff_df = pd.DataFrame(X_train.columns.delete(0))
coeff_df.columns = ['Feature']
coeff_df["Correlation"] = pd.Series(logreg.coef_[0])

coeff_df.sort_values(by='Correlation', ascending=False)

,Feature,Correlation
3,id_site,1.865240e-06
0,code_presentation,2.127671e-11
1,final_result,9.964395e-12
6,registration,6.843814e-12
5,week_from,3.999458e-12
7,total_clicks,3.557568e-12
4,activity_type,2.714062e-12
2,date_submitted,-1.749969e-11


In [32]:
svc = SVC()
svc.fit(X_train_r, Y_train_r)
Y_pred = svc.predict(X_test_r)
acc_svc = round(svc.score(X_train_r, Y_train_r) * 100, 2)
acc_svc

73.64

In [33]:
decision_tree = DecisionTreeClassifier()
decision_tree.fit(X_train_r, Y_train_r)
Y_pred = decision_tree.predict(X_test_r)
acc_decision_tree = round(decision_tree.score(X_train_r, Y_train_r) * 100, 2)
acc_decision_tree

78.61

In [34]:
# Create a StandardScaler instances
scaler = StandardScaler()

# Fit the StandardScaler
X_scaler = scaler.fit(X_train)

# Scale the data
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

X_train_scaled.shape

(146702, 9)

In [35]:
knn = KNeighborsClassifier(n_neighbors = 3)
knn.fit(X_train_r, Y_train_r)
Y_pred = knn.predict(X_test_r)
acc_knn = round(knn.score(X_train_r, Y_train_r) * 100, 2)
acc_knn

75.02

In [36]:
gaussian = GaussianNB()
gaussian.fit(X_train_r, Y_train_r)
Y_pred = gaussian.predict(X_test_r)
acc_gaussian = round(gaussian.score(X_train_r, Y_train_r) * 100, 2)
acc_gaussian

76.47

In [37]:
linear_svc = LinearSVC()
linear_svc.fit(X_train_r, Y_train_r)
Y_pred = linear_svc.predict(X_test_r)
acc_linear_svc = round(linear_svc.score(X_train_r, Y_train_r) * 100, 2)
acc_linear_svc

/Users/horsewell/anaconda3/envs/dev/lib/python3.10/site-packages/sklearn/svm/_classes.py:32: FutureWarning: The default value of `dual` will change from `True` to `'auto'` in 1.5. Set the value of `dual` explicitly to suppress the warning.
  warnings.warn(
/Users/horsewell/anaconda3/envs/dev/lib/python3.10/site-packages/sklearn/svm/_base.py:1242: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(


73.64

In [38]:
sgd = SGDClassifier()
sgd.fit(X_train_r, Y_train_r)
Y_pred = sgd.predict(X_test_r)
acc_sgd = round(sgd.score(X_train_r, Y_train_r) * 100, 2)
acc_sgd

73.64

In [39]:
decision_tree = DecisionTreeClassifier()
decision_tree.fit(X_train_r, Y_train_r)
Y_pred = decision_tree.predict(X_test_r)
acc_decision_tree = round(decision_tree.score(X_train_r, Y_train_r) * 100, 2)
acc_decision_tree

78.61

In [40]:
random_forest = RandomForestClassifier(n_estimators=100)
random_forest.fit(X_train_r, Y_train_r)
Y_pred = random_forest.predict(X_test_r)
random_forest.score(X_train_r, Y_train_r)
acc_random_forest = round(random_forest.score(X_train_r, Y_train_r) * 100, 2)
acc_random_forest

78.61

In [41]:
models = pd.DataFrame({
    'Model': ['Support Vector Machines', 'KNN', 'Logistic Regression', 
              'Random Forest', 'Naive Bayes', 
              'Stochastic Gradient Decent', 'Linear SVC', 
              'Decision Tree'],
    'Score': [acc_svc, acc_knn, acc_log, 
              acc_random_forest, acc_gaussian, 
              acc_sgd, acc_linear_svc, acc_decision_tree]})
models.sort_values(by='Score', ascending=False)

,Model,Score
3,Random Forest,78.61
7,Decision Tree,78.61
4,Naive Bayes,76.47
1,KNN,75.02
0,Support Vector Machines,73.64
2,Logistic Regression,73.64
5,Stochastic Gradient Decent,73.64
6,Linear SVC,73.64
